#**MoneyMoves!**
A Quiz based approach to retirement planning suggestions

In [12]:
## Run this cell once without comments to install packages, then comment them out.
!pip install openai
!pip install langchain
!pip install tiktoken
!pip install faiss-cpu

In [13]:
from langchain.chat_models import AzureChatOpenAI, ChatVertexAI
from langchain.embeddings import OpenAIEmbeddings, VertexAIEmbeddings
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.docstore.document import Document
from langchain.document_loaders import DirectoryLoader
from langchain.chains import AnalyzeDocumentChain
from langchain.chains.summarize import load_summarize_chain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
import os
os.environ["OPENAI_API_KEY"] = "be9bdecc8bf64e85bde69c04b2ad56f8"
os.environ["OPENAI_API_HOST"] = "https://tiaa-openai-azure-sweden.openai.azure.com/"
os.environ["OPENAI_API_EMBEDDING_KEY"] = "be9bdecc8bf64e85bde69c04b2ad56f8"
os.environ["OPENAI_API_EMBEDDING_HOST"] = "https://tiaa-openai-azure-sweden.openai.azure.com/"
os.environ["OPENAI_API_VERSION"] =  "2023-07-01-preview"
import openai
openai.api_type='azure'
openai.api_base = os.environ["OPENAI_API_HOST"]
openai.api_key = os.environ["OPENAI_API_KEY"]
#model_name='tiaa-gpt-4-32k'
base_model_name="tiaa-gpt-4"
chosen_llm = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=1
    )
embedding_model_name = 'tiaa-text-embedding-ada-002'
chosen_embedding_model = OpenAIEmbeddings(
        deployment=embedding_model_name,
        openai_api_base=os.environ["OPENAI_API_EMBEDDING_HOST"],
        openai_api_version= os.environ["OPENAI_API_VERSION"],
        openai_api_key=os.environ["OPENAI_API_EMBEDDING_KEY"],
        openai_api_type="azure",
        chunk_size=16
    )
finalAnswerTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=1,
        verbose = True
    )
condensingTheQuestionModel = AzureChatOpenAI(
        openai_api_base=os.environ["OPENAI_API_HOST"],
        openai_api_version=os.environ["OPENAI_API_VERSION"],
        deployment_name=base_model_name,
        openai_api_key=os.environ["OPENAI_API_KEY"],
        openai_api_type="azure",
        temperature=1,
        verbose = True
    )
memory = ConversationBufferMemory(
    output_key='answer',
    memory_key='chat_history',
    return_messages=True
    )


#**Core Logic**

Wrapping the output to display results in a suitable fashion.

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

The user is asked questions based on financial planning and retirement goals to get an understanding of their current standing as well as their future aspirations.

This provides us and the GenAI model an understanding of the person and what kind of financial planning would suit him/her.

In [14]:
questions = []

questions.append("Which of the following best describes you? a) I'm a meticulous planner, and I like to have a detailed financial roadmap for retirement.  b) I'm a free spirit, and I prefer a flexible approach to retirement planning. c) I'm risk-averse and prioritize the safety of my retirement investments. d) I'm uncertain")

questions = [
    ["q1) Which of the following categories do you fall under?",
    "1) I'm a Student",
    "2) I'm an Employee early into my career",
    "3) I'm a researcher",
    "4) I'm currently not employed. I may/may not be seeking a job at the moment."],
    ["q2) Which of the following best describes you?",
    "1) I'm a meticulous planner, and I like to have a detailed financial roadmap for retirement.",
    "2) I'm a free spirit, and I prefer a flexible approach to retirement planning.",
    "3) I'm risk-averse and prioritize the safety of my retirement investments.",
    "4) I'm uncertain"],
    ["q3) What age do you envision retiring at?",
     "1) 55 or earlier",
     "2) 60-65",
     "3) 66-70",
     "4) After 70"],
    ["q4) What is your primary retirement savings goal?",
     "1) Financial independence",
     "2) Travel and leisure",
     "3) Supporting family",
     "4) Leaving a legacy"],
    ["q5) How diligently do you contribute to your retirement savings currently?",
     "1) I consistently contribute the maximum allowed amount",
     "2) I regularly contribute, but not the maximum",
     "3) I contribute sporadically",
     "4) I haven't started saving for retirement yet"],
    ["q6) In your 20s, you suddenly receive a substantial amount of money. What's your immediate plan for this unexpected financial gain?",
     "1) Invest a substantial portion in a retirement account.",
     "2) Use it for a dream vacation or a significant purchase.",
     "3) Pay off existing debts, including student loans.",
     "4) I haven't considered it yet."]
]

res = []
ans = int()
for i in questions:
  print(i[0])
  print(i[1])
  print(i[2])
  print(i[3])
  print(i[4])
  ans = input("Choose between the options - 1, 2, 3 or 4 ")
  res.append((i[0]+" Ans: "+i[int(ans)]))

prompt ="Generate a financial plan using the given questionnaire and the answers"
msg = HumanMessage(content=prompt + "".join(res))
output=chosen_llm(messages=[msg])
# print(output.content)


q1) Which of the following categories do you fall under?
1) I'm a Student
2) I'm an Employee early into my career
3) I'm a researcher
4) I'm currently not employed. I may/may not be seeking a job at the moment.
Choose between the options - 1, 2, 3 or 4 4
q2) Which of the following best describes you?
1) I'm a meticulous planner, and I like to have a detailed financial roadmap for retirement.
2) I'm a free spirit, and I prefer a flexible approach to retirement planning.
3) I'm risk-averse and prioritize the safety of my retirement investments.
4) I'm uncertain
Choose between the options - 1, 2, 3 or 4 2
q3) What age do you envision retiring at?
1) 55 or earlier
2) 60-65
3) 66-70
4) After 70
Choose between the options - 1, 2, 3 or 4 2
q4) What is your primary retirement savings goal?
1) Financial independence
2) Travel and leisure
3) Supporting family
4) Leaving a legacy
Choose between the options - 1, 2, 3 or 4 1
q5) How diligently do you contribute to your retirement savings currently?

Based on the inferences made about the person's financial standing and future retirement aspirations, the LLM is used to determine what TIAA based retirement plan is to be suggested to the user.


The LLM has been instructed to pick from the following plans and provide reasoning for making it's choice.
The LLM was also fed TIAA's retirement plans and was made to provide key-words relating to each. This was used to infer similarities between the user's financial persona and the key-words.


```
1. Annuities and Investment Options
  - Financially Savvy
  - Risk-Tolerant
  - Strategic Planner
  - Long-Term Oriented
  - Goal-Driven
  - Investment-Minded
  - Open to Diversification
  - Patient Investor
  - Prudent Decision-Maker
  - Future-Focused
  
2. Employer-Sponsored Retirement Plans:
  - Employee Benefits-Conscious
  - Company-Loyal
  - Retirement-Ready
  - Team Player
  - Goal-Oriented
  - Responsible Saver
  - Cooperative
  - Long-Term Planner
  - Company Culture-Engaged
  - Tax-Efficient Investor

3. Individual Retirement Accounts (IRAs):
  - Independent Planner
  - Self-Directed Investor
  - Tax-Conscious
  - Financially Independent
  - Goal-Driven Saver
  - Future-Forward
  - Personal Responsibility
  - Long-Term Planner
  - Tax-Efficient Saver
  - Future-Focused

4. TIAA Advisory Services:
  - Seeker of Expertise
  - Informed Decision-Maker
  - Financially Literate
  - Seeking Guidance
  - Trusting
  - Willing to Learn
  - Goal-Oriented
  - Long-Term Planner
  - Open to Advice
  - Collaborative

5. Lifetime Income Options:
  - Risk-Averse
  - Stability-Seeking
  - Future-Focused
  - Long-Term Planner
  - Security-Conscious
  - Patient Investor
  - Annuity-Minded
  - Financially Cautious
  - Retirement-Ready
  - Steady Income-Seeking

6. Long-Term Care Insurance:
  - Prudent Planner
  - Health-Conscious
  - Future-Prepared
  - Risk-Aware
  - Safety-Minded
  - Responsible
  - Health-Considerate
  - Focused on Well-Being
  - Caring for the Future
  - Prepared for Contingencies

7. Education and Workshops:
  - Eager to Learn
  - Inquisitive
  - Proactive Learner
  - Financially Literate
  - Open to Education
  - Seeking Knowledge
  - Goal-Oriented Learner
  - Empowered
  - Committed to Learning
  - Future-Forward

8. Socially Responsible Investing:
  - Ethically Conscious
  - Environmentally Aware
  - Socially Committed
  - Values-Driven
  - Sustainable-Minded
  - Responsible Investor
  - Impact-Driven
  - Aligned with Values
  - Future-Forward
  - Socially Engaged

9. Customer Support and Service:
  - Customer-Centric
  - Relationship-Focused
  - Service-Oriented
  - Patient
  - Communicative
  - Support-Seeking
  - Trusting
  - Open to Guidance
  - Seeking Assistance
  - Empathetic

10. Legacy Planning and Estate Services:
  - Forward-Thinking
  - Family-Oriented
  - Responsible Steward
  - Goal-Oriented Planner
  - Values Family and Legacy
  - Estate-Conscious
  - Willing to Plan Ahead
  - Legacy-Driven
  - Future-Focused
  - Responsible for Generations
```


In [15]:
suggestion_prompt = "Suggest strictly two best suited TIAA plans with explanation from the following - Annuities and Investment Options,Employer-Sponsored Retirement Plans, Individual Retirement Accounts (IRAs),TIAA Advisory Services,Lifetime Income Options,Long-Term Care Insurance,Education and Workshops,Socially Responsible Investing,Customer Support and Service,Legacy Planning and Estate Services from the given result: "+str(output.content)
msg = HumanMessage(content=suggestion_prompt)
output=chosen_llm(messages=[msg])
print(output.content)

1) Individual Retirement Accounts (IRAs): Given your current status of unemployment and preference for a flexible approach to retirement planning, setting up either a Traditional or Roth IRA with TIAA would be beneficial. These plans would allow you to make contributions to your retirement account on your own schedule and would also provide tax advantages. The money in your IRA can be invested in a number of different ways, including in stocks and bonds, which would help your savings grow over time. Moreover, IRAs are ideal for people who don't have access to an employer-sponsored retirement plan.

2) TIAA Advisory Services: As a person just starting out with planning retirement savings, the TIAA Advisory Services will be very helpful. They offer personalized investment advice and can help you to develop a financial strategy that suits your long-term goals and risk tolerance. With your preference for flexibility, you would likely benefit from working with an advisor to explore a range 